Dataset from: https://www.kaggle.com/areeves87/rscience-popular-comment-removal

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

In [ ]:
#encoding = 'latin-1'
df = pd.read_csv('reddit_200k_train.csv', encoding="ISO-8859-1")
df.head()

In [ ]:
valid_pct = 0.2
cut = int(valid_pct * len(df)) + 1
data_lm = (TextLMDataBunch.from_df(path=".",
                                 train_df = df[cut:],
                                 valid_df = df[:cut],
                                 text_cols = "body",
                                 label_cols = "REMOVED")
       )

In [ ]:
data_lm.save()

In [ ]:
data_lm = load_data(".")

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.load('fit_head');

In [ ]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned')

In [ ]:
TEXT = "Newton's first law has nothing to do with"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
!jupyter nbconvert rscience-comment-removal.ipynb --to html --output nbs/2019-07-02_lm_only

In [ ]:
valid_pct = 0.2
cut = int(valid_pct * len(df)) + 1
data_clas = (TextClasDataBunch.from_df(path=".",
                                 train_df = df[cut:],
                                 valid_df = df[:cut],
                                 text_cols = "body",
                                 label_cols = "REMOVED")
       )
data_clas.save('data_clas.pkl')

In [ ]:
data_clas.save('data_clas.pkl')

In [ ]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [ ]:
data_clas.show_batch(10)

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-2, moms=(0.8,0.7))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('first')

In [ ]:
learn.load('first');

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')

In [ ]:
learn.load('second');

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('third')

In [ ]:
learn.load('third');

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

# Let's Play

From https://www.reddit.com/r/science/wiki/rules#wiki_comment_rules

## Comment Rules

Please report any inappropriate comments to the moderators. If you feel your comment was mistakenly removed, please message the moderators and include the link to your comment.
1. No off-topic comments, memes, or jokes

Comments must be on topic and not a meme or joke. Comments should constructively contribute to the discussion or be an attempt to learn more.
2. No abusive, offensive, or spam comments

Abusive, offensive, or spam comments will be subject to removal and repeat or malicious offenders will be banned. Bots and novelty accounts are prohibited and will be banned immediately.
3. No personal anecdotes

Comments that only rely on a user's non-professional anecdotal evidence to confirm or refute a study will be removed (e.g. "I do that but that result doesn't happen to me"). Comments should be limited in personal details and scientific in nature. Including references to peer-reviewed research to support your claims is highly encouraged.
4. Comments dismissing established science must provide evidence

Comments that dispute well-established scientific concepts (e.g. gravity, vaccination, anthropogenic climate change, etc.) must be supported with appropriate peer-reviewed evidence. Links to personal blogs or 'skeptic' websites are not valid forms of evidence. Comments that are overtly fringe and/or unsubstantiated will be removed.
5. No medical advice

Offering or seeking medical advice is strictly prohibited and offending comments will be removed. Discussions regarding the advantages and/or disadvantages of certain treatments, diets, or supplements is allowed as long as relevant and reputable evidence is provided.
6. Repeat or flagrant offenders will be banned

Users that frequently or flagrantly violate the comment rules will be banned.

In [ ]:
comments = [
    "Nice try FBI.", #1 meme
    "You have no idea wtf you're talking about.", #2 abusive/offensive
    "I do that but that result doesn't happen to me", #3 personal anecdote (verbatim example)
    "No you have to flip it. If you don't believe me, just google it." #3 anecdote (something my mom would say LOL)
    "My IQ is 200. I'm pretty sure I'm right.",
    "Thank goodness my daughter wasn't vaccinated or she would be autistic too.", #4
    "Get a prescription for Benzaclin. It will clear up your acne.", #5 medical advice (though I do use benzaclin myself)
    "Sorry, my last comment was uncalled for. I was too caught up in the argument.",
    "Actually, several groups from UCLA failed to reproduce those results, and that paper ended up being retracted last year.",
    #these are copy and pasted comments.
    "I’d love to see a study looking at how many people who quit drinking also make other positive lifestyle choices leading to that feeling of how everything’s better than before.",
    "The wording of the summary implies that male moderate drinkers who quit alcohol were mentally worse off or unchanged.",
    "Pretty moderate in Europe.",
    "I’ll drink to that",
    "So we can't cure the common cold or cancer, but maybe one can cure the other?",
    "That sounds pretty cool actually",
    "Yeah, cough and post nasal drip is definitely preferable to blood in my urine, death etc...",
    "Ugh. I seriously hate post nasal drip. Keeps me up at night when I got it",
    "I have it right now and just crave a good night's rest"
]

In [ ]:
for comment in comments:
    pred = learn.predict(comment)
    print(comment)
    print("Moderator Removal Probability: " + str(pred[2][1].item()))
    print("\n")

In [ ]:
!jupyter nbconvert rscience-comment-removal.ipynb --to html --output nbs/2019-07-08_clas_0.712acc